# Advent of Code 2021 - Day 2
> My solution and thoughts

- toc: false
- badges: false
- comments: false
- categories: [advent-of-code]

# Day 2!
Technically, I wrote this the same day as day 1, and on the third day release... but you can forgive that, right?

## Part 1
Anyway, we're on to day 2! This challenge centers around calculating positions. So our input is a list of directions, and values to apply to those directions. First things first, we're going to want to split those up into something meaningful. Personally, I love a good `namedtuple`:

In [21]:
from collections import namedtuple

Movement = namedtuple('Movement', ['direction', 'value'])

So each instance in the input list can now be instantiated as a `Movement`, so let's read in that data!

In [22]:
with open('../assets/inputs/aoc/2021/day_2.txt', 'r') as data_file:
    lines = data_file.readlines()
    movements = [Movement(line.strip().split(' ')[0], int(line.strip().split(' ')[1])) for line in lines]
    
display(movements[:3])
display(movements[997:])

[Movement(direction='forward', value=4),
 Movement(direction='down', value=8),
 Movement(direction='down', value=3)]

[Movement(direction='forward', value=4),
 Movement(direction='forward', value=4),
 Movement(direction='forward', value=8)]

Ok! We have some data, and we've got it formatted in a way that seems reasonable. Essentially, we need to calculate the horizontal and vertical position that the list of movements leaves us in. We will subtract from the vertical when we go up, add when we go down, and add to the horizontal when we move forward. This is a pretty easy set of conditions:

In [23]:
horizontal_position = 0
vertical_position = 0

for movement in movements:
    if movement.direction == 'up':
        vertical_position -= movement.value
    elif movement.direction == 'down':
        vertical_position += movement.value
    else:
        horizontal_position += movement.value

display(horizontal_position * vertical_position)

1648020

Since the puzzle asks us to give the answer as the multiple of our tracked values, we do.

Success!

But, what about that code? Does it look... a little gross? I think it does. Namely, we're doing almost the exact same thing for every branch. Can we clean that up?

One approach that I like is to declare the operations in a dictionary:

In [24]:
import operator as ops

movement_operation = {
    'up': ops.sub,
    'down': ops.add,
    'forward': ops.add
}

Now we can re-write a bit:

In [25]:
horizontal_position = 0
vertical_position = 0

for movement in movements:
    op = movement_operation[movement.direction]
    if movement.direction in ['up', 'down']:
        vertical_position = op(vertical_position, movement.value)
    else:
        horizontal_position = op(horizontal_position, movement.value)

display(horizontal_position * vertical_position)

1648020

Is this better? It's a little harder to read, and we have now encoded that `vertical_position` has to be the thing that both 'up' and 'down' deal with. If we were to need to change 'down' to deal with some other value, this is harder to break apart. On the other hand, the operations are now lifted. If I need to change the operation that is performed for anything, I can do so in one place and be sure that it propogates to all the usage points. That's pretty cool.

## Part 2
Now we introduce the idea of "aim". This is fun if you're a person who has played _entirely_ too much [Subnautica](http://subnauticagame.com/) because the idea makes sense. In this story, you're piloting a sub-marine vehicle. So the aim is basically where the nose of the craft is pointed. Adjusting 'up' and 'down' now affects aim, and only forward changes the values. Pretty easy:

In [26]:
aim = 0
horizontal_position = 0
vertical_position = 0

for movement in movements:
    op = movement_operation[movement.direction]
    if movement.direction in ['up', 'down']:
        aim = op(aim, movement.value)
    else:
        horizontal_position = op(horizontal_position, movement.value)
        vertical_position = op(vertical_position, (movement.value * aim))

display(horizontal_position * vertical_position)

1759818555

Ok, so we solved it, that's cool... But is this the best way?

We iterate the data (in its cleaned state) just the once. So that's good, but it still feels like we're missing something. It seems like the answer being the two values multiplied together means something.

Maybe I'll figure it out at some point...